In [3]:
import numpy as np 
import pandas as pd

In [4]:
def get_compare_matrix(pair_criteria_file: str):
    # Read the lower triangle matrix from the input file
    with open(pair_criteria_file, 'r') as file:
        lines = file.readlines()
        matrix_size = len(lines)
        lower_triangle = np.zeros((matrix_size, matrix_size), dtype=float)
        for i in range(matrix_size):
            row = list(map(float, lines[i].strip().split()))
            for j in range(i + 1):
                lower_triangle[i, j] = row[j]
        
    for i in range(len(lower_triangle)):
        for j in range(len(lower_triangle[i])):
            if lower_triangle[i,j] == 0:
                lower_triangle[i,j] = 1 / lower_triangle[j,i]
    return lower_triangle

def calculate_normalize_matrix(matrix):
    matrix_norm = np.zeros_like(matrix)
    for j in range(matrix.shape[1]):
        column_sum = np.sum(matrix[:, j])
        matrix_norm[:, j] = matrix[:, j] / column_sum
    return matrix_norm

def calculate_weight_matrix(matrix):
    matrix_weight = np.zeros((matrix.shape[0], 1))
    for i in range(matrix_weight.shape[0]):
        matrix_weight[i] = sum(matrix[i]) / len(matrix[i])

    return matrix_weight

In [5]:
def normalize_matrix(matrix):
    # Calculate the sum of each column
    column_sums = np.sum(matrix, axis=0)
    # Divide each element in a column by its sum
    normalized_matrix = matrix / column_sums
    
    return normalized_matrix

def get_worst_best_criteria():
    best_c = []
    worst_c = []
    for col in range(norm_matrix.shape[1]):
        if criteria_importance[col] == "min":
            best_value = np.min(norm_matrix[:, col])
            worst_value = np.max(norm_matrix[:, col])
    
        if criteria_importance[col] == "max":
            best_value = np.max(norm_matrix[:, col])
            worst_value = np.min(norm_matrix[:, col])

        best_c.append(best_value)
        worst_c.append(worst_value)
            
    best_c = np.array(best_c)
    worst_c = np.array(worst_c)

    return (best_c, worst_c)
def get_utility():
    utility = np.zeros(alternative_size)

    for i in range(len(utility)):
        temp = 0 
        for j in range(len(weight_vector)):
            temp += weight_vector[j] * (best_criterias[j]-norm_matrix[i,j]) / (best_criterias[j]-worst_criterias[j])
        utility[i] = temp 
    return utility

def get_regret():
    regret = np.zeros(alternative_size)

    for i in range(len(regret)):
        temp = []
        for j in range(len(weight_vector)):
            value = weight_vector[j] * (best_criterias[j]-norm_matrix[i,j]) / (best_criterias[j]-worst_criterias[j])
            temp.append(value)
        regret[i] = np.max(temp) 
    return regret

def get_Q_indices():
    Q_indices = np.zeros(alternative_size)
    Q_indices = v * (utility - max_utility) / (max_utility - min_utility) \
            + (1 - v) * (regret - max_regret) / (max_regret - min_regret)

    return Q_indices

# Init the purpose of each criteria. For example: cost should be minimized and benefit should be maximized
criteria_importance = ["max", "max", "max", "max", "max"]
data = pd.read_csv("data.csv")
# create decision matrix
decision_matrix = data.values

# vikor method with the weight as input manually
# ensure sum of all weights = 1
# weight = pd.read_csv("weight_VIKOR.csv")
#weight_vector = weight.values[0]
### get compare matrix from input file (only with low triangle)
compare_matrix = get_compare_matrix("weight.txt")
normalized_matrix = calculate_normalize_matrix(compare_matrix)
# sum of weight should be 1. weight for each criterion
weight_vector = calculate_weight_matrix(normalized_matrix)
weight_vector = weight_vector.flatten()

# normalize decision matrix
norm_matrix = normalize_matrix(decision_matrix)

# get max/min value for each criteria
best_criterias, worst_criterias = get_worst_best_criteria()

# calculate utility and regret values
alternative_size = decision_matrix.shape[0]
utility = get_utility()
regret = get_regret()

max_utility = np.max(utility)
min_utility = np.min(utility)
max_regret = np.max(regret)
min_regret = np.min(regret)

# calculate Q indices
v = 0.5 # parameter: 0.5 means consider utility and regret are equal
Q_indices = get_Q_indices() 
# Q_indices

In [6]:
# display results 
data['utility (S)'] = utility
data['regret (R)'] = regret
data['Q_index'] = Q_indices

data.sort_values("Q_index", ascending=True, inplace=True)
raking = [(i+1) for i in range(alternative_size)]
data['ranking'] = raking

# rows are alternatives
# columns are criterias
data.head(10)

,Python,Sql,ML,Tableau,Excel,utility (S),regret (R),Q_index,ranking
14,0.88,0.83,0.93,0.19,0.63,0.205843,0.071376,-0.996338,1
159,0.83,0.85,0.71,0.37,0.55,0.262753,0.069433,-0.956336,2
1,0.81,0.90,0.62,0.71,0.92,0.211270,0.090982,-0.955215,3
89,0.70,0.82,0.92,0.52,0.93,0.211179,0.100399,-0.937531,4
144,0.84,0.91,0.91,0.05,0.26,0.254916,0.083712,-0.935430,5
117,0.76,0.95,0.77,0.09,0.62,0.265340,0.080319,-0.933830,6
77,0.75,0.86,0.83,0.32,0.58,0.259739,0.083666,-0.931818,7
197,0.95,0.65,0.62,0.74,0.61,0.253421,0.090982,-0.922874,8
70,0.84,0.58,0.96,0.44,0.75,0.237778,0.100814,-0.916341,9
34,0.73,0.99,0.84,0.90,0.01,0.239901,0.100022,-0.916205,10
